# Make individual and diff images of 2 versions of CETB files

In [ ]:
%matplotlib inline
import glob
import matplotlib.pyplot as plt
import math
from netCDF4 import Dataset
import os
import numpy as np
import re

In [ ]:
#%cd /projects/PMESDR/vagrant/NSIDCtest/vmwintryhummock
%cd /projects/brodzik/v1.3images
%ls

In [ ]:
yyyyddd = 1979079 # SMMR
#yyyyddd = 1988080 # F08
#yyyyddd = 1991079 # F10
#yyyyddd = 1992080 # F11
#yyyyddd = 1996080 # F13
#yyyyddd = 1998079 # F14
#yyyyddd = 2006079 # F15
#yyyyddd = 2016080 # F16 (late in sensor lifetime, to examine new ltods)
#yyyyddd = 2009079 # F17
#yyyyddd = 2010079 # F18
#yyyyddd = 2015079 # F19
#yyyyddd = 2003079 # AMSR-E
#os.mkdir("%s" % yyyyddd)
#os.mkdir("%s/both" % yyyyddd)
#os.mkdir("%s/diff" % yyyyddd)
%ls

In [ ]:
imgtype = "37H"
sensor = "*"
oldList = sorted(glob.glob("/projects/brodzik/v1.2/%s/*%s*%s*%s*nc" % (yyyyddd, sensor, yyyyddd, imgtype)))
#oldList = sorted(glob.glob("/projects/brodzik/v1.2/%s/*%s*%s*nc" % (yyyyddd + 1, yyyyddd + 1, imgtype)))
newList = sorted(glob.glob("/projects/brodzik/v1.3/%s/*%s*%s*%s*nc" % (yyyyddd+1, sensor, yyyyddd+1, imgtype)))
#newList = sorted(glob.glob("/projects/brodzik/v1.3/%s/*%s*%s*nc" % (yyyyddd + 1, yyyyddd + 1, imgtype)))
print('\n'.join(oldList))
print("")
print('\n'.join(newList))

In [ ]:
def read_cetb(file, varname="TB"):
    fid = Dataset(file, 'r', format='NETCDF4')
    data = np.squeeze(fid.variables[varname][:])
    fid.close()
    print(file)
    print(np.shape(data))
    print(np.amin(data), np.amax(data))
    return(data)   

In [ ]:
def show_both(file1, file2, yyyyddd, varname="TB"):
    img1 = read_cetb(file1,varname=varname)
    img2 = read_cetb(file2,varname=varname)
    
    if re.search(r'EASE2_T', file1):
        fig, ax = plt.subplots(2, 1, figsize=(9.5,9))
        imgtype = 'T'
    else:
        fig, ax = plt.subplots(1, 2, figsize=(10,6))
        imgtype = 'NS'
    
    if varname == "TB":
        vmin = 50.
        vmax = 300.
    else: # assume TB_TIME
        if 'T' == imgtype:
            vmin = 0.
            vmax = 1440.
        else:
            vmin = -720.
            vmax = 1440.+720.

    ax[0].imshow(img1, cmap=plt.cm.gray, interpolation='None', vmin=vmin, vmax=vmax)        
    ax[0].set_title("%s: v1.2" % varname)
    ax[0].axis('off')
    ax[1].imshow(img2, cmap=plt.cm.gray, interpolation='None', vmin=vmin, vmax=vmax)
    ax[1].set_title("%s: v1.3" % varname)
    ax[1].axis('off')
    
    fileName = os.path.basename(file2)
    fig.suptitle(fileName)
    fig.tight_layout()
    fig.savefig("%s/both/%s.%s.both.png" % (yyyyddd, fileName, varname))
    return(fig)

In [ ]:
varname = "TB_time"
old_file_index = 2
new_file_index = 2
fig = show_both(oldList[old_file_index], newList[new_file_index], yyyyddd, varname=varname)

In [ ]:
varname = "TB"
old_file_index = 5
new_file_index = 4
fig = show_both(oldList[old_file_index], newList[new_file_index], yyyyddd, varname=varname)

In [ ]:
close('all')
#fig.savefig("NSIDC-0630-EASE2_N3.125km-F16_SSMIS-2005333-91H-M-SIR-CSU-v1.3.TB.both.png")

In [ ]:
def show_diff(file1, file2, yyyyddd, varname="TB", verbose=False):
    img1 = read_cetb(file1,varname=varname)
    img2 = read_cetb(file2,varname=varname)
    
    diff = img2 - img1

    rows, cols = np.shape(diff)
    print("%s - %s" % (file2, file2))
    print(np.shape(diff))
    print(np.amin(diff), np.amax(diff))
    
    fig, ax = plt.subplots(1, 1, figsize=(8,8))
    im = ax.imshow(diff, cmap=plt.cm.BrBG, interpolation='None', vmin=-1., vmax=1.)
    ax.set_title('%s: v1.3 - v1.2' % varname)
    ax.axis('off')
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    
    fileName = os.path.basename(file2)
    fig.suptitle(fileName)
    fig.tight_layout()
    fig.savefig("%s/diff/%s.%s.diff.png" % (yyyyddd, fileName, varname))
    
    diff[ diff==0 ] = np.nan
    
    sigdiffs = diff[np.isnan(diff) == False]
    num_diffs = np.shape(sigdiffs)[0]
    print("The number of non-zero pixels is %s" % num_diffs)
    
    print("This represents                  %.3f %% of the pixels in the image" % (
        100. * np.float(num_diffs) / np.float( rows * cols ) ))
    if verbose:
        print("The non-zero pixel values are:")
        print(sigdiffs)
    
    return(fig)

In [ ]:
fig = show_diff(oldList[old_file_index], newList[new_file_index], yyyyddd, varname="TB")

# use this to loop through all files for one day

In [ ]:
print( len(newList), len(oldList))

In [ ]:
#arange(len(oldList))[:2]

In [ ]:
for i in arange(len(oldList)):
    for varname in ['TB', 'TB_time']:
        fig = show_both(oldList[i], newList[i], yyyyddd, varname=varname)
        
    fig = show_diff(oldList[i], newList[i], yyyyddd)
    #close('all')